# Pinecone Quickstart

## 1. Load API Key with .env

In [ ]:
from dotenv import load_dotenv

# Your .env file should contain PINECONE_API_KEY=abc
load_dotenv()

## 2. Initialize Pinecone client

Next, use your API key to initialize your client:

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

## 3. Prepare language model for vector encoder

We use a small transformers language model to create 364-dimensional embeddings. You can out models for generating embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = embedder.encode("Example sentences.")
dimension = embeddings.shape[0]

## 4. Create Pinecone

This creates a serverless index named "quickstart" that performs nearest-neighbor search using the Euclidean distance metric for your vectors

In [ ]:
index_name = "quickstart"
pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="euclidean",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

## 5. Generate vector values from wikipedia text

We retrieve a wikipedia based dataset with Hugging Face's datasets library. Note that this dataset contains Cohere's vectors, but we're generating our own in this notebook.

In [ ]:
from datasets import load_dataset
import torch
import cohere
from tqdm.auto import tqdm

max_docs = 1000 # Increase to use etnire dataset
docs_stream = load_dataset(f"Cohere/wikipedia-22-12-simple-embeddings", split="train", streaming=True)

vectors = []

pbar = tqdm(total=max_docs)
for doc in docs_stream:
    vec = {
        "id": str(doc["id"]),
        "values": embedder.encode(doc["title"] + " " + doc["text"]).tolist(),
        "metadata": {
            "title": doc["title"],
            "text": doc["text"],
            "url": doc["url"],
        }
    }
    vectors.append(vec)
    pbar.update(1)
    if len(vectors) >= max_docs:
        break


## 6. Upsert vectors

Now that you’ve created your index and the vector embeddings of your wikipedia data, you can upsert these vectors into your index.

In [ ]:
index = pc.Index(index_name)

index.upsert(
    vectors=vectors,
    namespace="ns1"
)


## 7. Check the index

Pinecone is eventually consistent, so there can be a delay before your upserted vectors are available to query. Use the describe_index_stats operation to check if the current vector count matches the number of vectors you upserted:

In [ ]:
index.describe_index_stats()

## 8. Run a similarity search

In [ ]:
query = "What's the greatest human advancement of all time?"

def search(query: str):
    query_results = index.query(
        namespace="ns1",
        vector=embedder.encode(query).tolist(),
        include_metadata=True,
        top_k=3,
    )
    matches = query_results.get("matches", [])
    if matches:
        return matches[0]["metadata"]
    else:
        return {}

search(query)

## 9. Deploy an app to port forward and share publically

In [ ]:
import gradio as gr

gr.close_all()

iface = gr.Interface(
    fn=search,
    inputs=gr.Textbox(lines=2, placeholder="Type your query here..."),
    outputs="json",
    title="Search Wikipedia text with Pinecone Index",
    description="This is a simple wikipedia search engine powered by transformers and pinecone",
)

# Run the Gradio app on localhost:5000 or whichever port you specified
iface.launch(server_port=5000, inline=False, quiet=True)

print(f"See your app deployed publically with the port you're securely forwarding: {os.environ.get('PORT_FORWARD_URL')}")

## 10. Clean up

When you no longer need the index, use the delete_index operation to delete it.

In [ ]:
pc.delete_index(index_name)